# **Sharing an Optimized Part of the Dataset**

Alter's goal is to obtain the most possible information, while Ego aims to hide its path.

The shared data, therefore, is an (optimal) deal between the two goals. However, making such deal is far from trivial.

In this notebook, we implement an optimization strategy inspired by the well-known Zeuthen negotiation strategy. In this implementation, Ego will make the calculations, also considering the needs of Alter.

In [1]:
import numpy as np
import pandas as pd

import json

from multiprocessing.pool import Pool

from tqdm.notebook import trange

import sys
sys.path.append("05_zopt_helpers")
import zopt

In [2]:
SEEDS = [42, 1234, 1867, 613, 1001]
TIME_LIMIT = 300
FALL_BACK_TIME = 15 #seconds -> corresponds to max. 208.3 m

In [11]:
RESULTS_ROOT = "../../02_data/01_simulation_results/"
VEH_LIST_PATH = "../../02_data/veh_list.json"
MEETING_PATH = "../../02_data/03_meeting_data/"
COMBINED_PATH = "../../02_data/03_meeting_data/combined_dataset.csv"

SOURCE_PROB_PATH = "../../02_data/source_probs.json"
FORWARD_PROB_PATH = "../../02_data/forward_probs.json"
PARKING_DEFINITION_FILE = "../../01_simulation/02_scenario/parking_areas.add.xml"

MEETING_VEHICLES = "../../02_data/meeting_vehicles"

In [4]:
import os


if not(os.path.exists(f"{MEETING_PATH}/zopt")):
    os.makedirs(f"{MEETING_PATH}/zopt")
for s in SEEDS:
    if not(os.path.exists(f"{MEETING_PATH}/zopt/{s}")):
        os.makedirs(f"{MEETING_PATH}/zopt/{s}")

In [6]:
#reading *test* vehicles:

with open(VEH_LIST_PATH) as f:
    veh_list  = json.load(f)

test_vehicles = veh_list["test_vehs"]

In [7]:
p_data = pd.read_csv(COMBINED_PATH)
p_data["time"] = p_data["time"].astype(int)
p_data = p_data[p_data["veh_id"].isin(test_vehicles)]
receive_time = [-1]*len(p_data)
p_data["receive_time"] = receive_time

In [8]:
def combine_commuters(veh_id):
    if veh_id.startswith("carIn"):
        return veh_id.split(":")[0]
    return veh_id

In [10]:
with open(SOURCE_PROB_PATH) as f:
    source_probs = json.load(f)

with open(FORWARD_PROB_PATH) as f:
    forward_probs = json.load(f)

In [12]:
parking_df = pd.read_xml(PARKING_DEFINITION_FILE, xpath="parkingArea")
parking_df = parking_df.set_index("id")

In [ ]:
p_edges = []
for _,r in parking_df.iterrows():
    p_edges.append(r["lane"].split("_")[0])
parking_df["edge"] = p_edges

meas_edge = []
for _,r in p_data.iterrows():
    meas_edge.append(parking_df.loc[r.parking_id].edge)
p_data["edge"] = meas_edge

In [11]:
def collect_meeting_vehicles(m_data, meeting_time, seed, meeting_times, meeting_time_gap=TIME_LIMIT):
    def create_meeting_dict(sender, receiver, direction, receiver_position):
        return {
            "sender": sender,
            "receiver": receiver,
            "receiver_position": receiver_position,
            "direction": direction
        }
    
    #collecting recently met vehicles:
    meetings = m_data[m_data["time"] == meeting_time]
    meetings = meetings[meetings["seed"] == seed]

    mets = []

    #same edges:
    for edge in edge_to_idx:
        vehs = meetings[meetings["edge"] == edge]["veh_id"].unique()
        for i in range(len(vehs)):
            for j in range(i+1, len(vehs)):
                sender = vehs[i]
                receiver = vehs[j]
                #if not met yet or met long time ago:
                if ((not((sender, receiver) in meeting_times)) or
                    meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                    mets.append(create_meeting_dict(sender, receiver, "same", edge))
                    mets.append(create_meeting_dict(receiver, sender, "same", edge)) #they meet vice-versa

    #opposed edges:
    for edge in edge_to_idx:
        #only "reversed" edges are processed, to avoid duplicated meetings:
        if edge.startswith("-"):
            veh_edge = meetings[meetings["edge"] == edge]["veh_id"].unique()
            contra_edge = edge.split("-")[1]
            veh_contra = meetings[meetings["edge"] == contra_edge]["veh_id"].unique()
            for sender in veh_edge:
                for receiver in veh_contra:
                    #if not met yet or met long time ago:
                    if ((not((sender, receiver) in meeting_times)) or
                        meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                        mets.append(create_meeting_dict(sender, receiver, "opposed", contra_edge))
                        mets.append(create_meeting_dict(receiver, sender, "opposed", edge)) #they meet vice-versa


    return mets #vehicles at the same time, at the same place, not the 'ego' vehicle and not met recently

In [12]:
def collect_data_upon_meeting(senders_data, receiver_position, on_same_edge, meeting_time, seed, time_limit=TIME_LIMIT, ego_time=FALL_BACK_TIME):
    send_data = senders_data[senders_data["seed"] == seed]
    send_data = send_data[send_data["time"] <= meeting_time]
    send_data = send_data[send_data["time"] >= meeting_time-time_limit]
    
    if len(send_data)==0:
        return None, None

    #calculating the zopt solution:
    probabilities = source_probs if on_same_edge else forward_probs
    s_data, perf = zopt.calc_send_data(send_data, probabilities[str(edge_to_idx[receiver_position])], idx_to_edge,
                                       receiver_position, meeting_time)
    has_alters = True if (s_data is None) or (len(s_data) == 0) else False
    s_data = pd.concat([s_data, send_data[send_data["time"] >= meeting_time-ego_time]],
                            ignore_index=True)
      
    if s_data is None:
        return None, None
    perf["has_no_zopt_data"] = has_alters
    return s_data.drop(columns=["receive_time"]), perf

In [13]:
def receive_data(args):
    sender, receiver, time, seed, vehicles_kb, receiver_position, on_same_edge = args

    rec_dat, perf = collect_data_upon_meeting(vehicles_kb[sender], receiver_position,
                                              on_same_edge, time, seed)
    if rec_dat is None:
        store_shared_data = {
            "sender": sender,
            "receiver": receiver,
            "time": time,
        }
        updated_kb = vehicles_kb[receiver]
        
    else:     
        received_data = rec_dat.copy()

        #storing shared data:
        store_shared_data = {
            "sender": sender,
            "receiver": receiver,
            "time": time,
            "data": received_data.to_json(),
            "performance": perf
        }

        #fusing data into the receiver vehicle's dataset:
        rec_t = [time]*len(received_data)
        received_data["receive_time"] = rec_t

        updated_kb = pd.concat([vehicles_kb[receiver], received_data], ignore_index=True)
        updated_kb = updated_kb.drop_duplicates(subset="hash", ignore_index=True)

    return store_shared_data, updated_kb

In [ ]:
from importlib import reload
reload(zopt);

def per_seed_script(seed):
    ps_data = p_data[p_data["seed"] == seed]
    
    meeting_times = {}
    vehicles_kb = {}
    store_sharing = []

    for veh in test_vehicles:
        vehicles_kb[veh] = ps_data[ps_data["veh_id"] == veh].copy()

    for t in trange(min(p_data["time"]), max(p_data["time"])):
        meetings = collect_meeting_vehicles(m_data, t, seed, meeting_times)
        for meet_map in meetings:
            sender = meet_map["sender"]
            receiver = meet_map["receiver"]
            receiver_position = meet_map["receiver_position"]
            on_same_edge = meet_map["direction"] == "same"

            arguments = [sender, receiver, t, seed, vehicles_kb,
                         receiver_position, on_same_edge]
            meeting_times[(sender, receiver)] = t

            new_store, updated_kb = receive_data(arguments)
            vehicles_kb[receiver] = updated_kb
            store_sharing.append(new_store)

    for veh in vehicles_kb:
        vehicles_kb[veh].to_csv(f"08_shared_veh_data/zopt/{seed}/{veh}.csv", index=False)
    store_dict = {
        "shared_data": store_sharing
    }
    with open(f"08_shared_veh_data/zopt/{seed}/shared_data.json", "w") as f:
        json.dump(store_dict, f)

with Pool(5) as ps:
    ps.map(per_seed_script, SEEDS)